# Pré-processamento para interpolação de voxel
* Todos os conjuntos de dados foram pré-processados para obter o tamanho de voxel isométricos de (1x1x1). Isso foi feito usando a biblioteca SimpleITK, pois esta permite mudar o tamanho do voxel em precisar manupular manualmente a affine dos volumes. Essa biblioteca tem algumas funções que faz isso automaticamente, garantindo a compatibilidade dos dados para o treinamento. As imagens e as segmentações passaram por esse pré-processamento. Alguns volumes dos conjuntos de dados do CC359 já tem os voxels isométricos e um desses foi usado como referência para aplicar nos outros. 

In [1]:
from scipy.ndimage import zoom
import SimpleITK as sitk
import nibabel as nib
import subprocess
import numpy as np
import glob
import os

def save_nii_zoom(input_paths: str, ref_path: str):
    '''
    Documentação
    '''
    for input_path in input_paths:
        T1_file = 'brain_T1w.nii.gz'
        sitk_T1 = sitk.ReadImage(os.path.join(input_path, T1_file))
        origin = sitk_T1.GetOrigin()
        spacing = sitk_T1.GetSpacing()
        directions = sitk_T1.GetDirection()

        mask_T1_file = 'fix_brainmask_T1w.nii.gz'
        sitk_mask_T1 = sitk.ReadImage(os.path.join(input_path, mask_T1_file))

        pre_affine = nib.load(os.path.join(input_path, T1_file)).affine
        print(f"Pre-affine {pre_affine}")  

        T1_array = sitk.GetArrayFromImage(sitk_T1)
        mask_T1_array = sitk.GetArrayFromImage(sitk_mask_T1)

        input_spacing = np.array(spacing[::-1])  # 1, 1, 2 -> 2, 1, 1
        target_spacing = np.array(sitk.ReadImage(ref_path).GetSpacing())[::-1]
        print(f"Transforming spacing of {T1_array.shape} from {input_spacing} to {target_spacing}")

        T1_iso_array = zoom(T1_array, input_spacing/target_spacing)
        mask_T1_iso_array = zoom(mask_T1_array, input_spacing/target_spacing, order=0)
        print(f"New shape: {T1_iso_array.shape}")

        iso_T1 = sitk.GetImageFromArray(T1_iso_array)
        iso_T1.SetSpacing(target_spacing[::-1])
        iso_T1.SetOrigin(origin)
        iso_T1.SetDirection(directions)
        iso_mask_T1 = sitk.GetImageFromArray(mask_T1_iso_array)
        iso_mask_T1.SetSpacing(target_spacing[::-1])
        iso_mask_T1.SetOrigin(origin)
        iso_mask_T1.SetDirection(directions)



        T1_out_name = "iso_brain_T1w.nii.gz"
        mask_T1_out_name = "iso_brainmask_T1w.nii.gz"

        sitk.WriteImage(iso_T1, os.path.join(input_path, T1_out_name))
        sitk.WriteImage(iso_mask_T1, os.path.join(input_path, mask_T1_out_name))
        post_affine = nib.load(os.path.join(input_path, T1_out_name)).affine
        print(f"Post-affine {post_affine}")

        print("Done")

    return iso_T1, iso_mask_T1

In [2]:
ref_path = "/home/joany/dataset_train/val/CC0126/brain_T1w.nii.gz"
folder_mri = f"/home/joany/dataset_train/val"
input_paths = glob.glob(os.path.join(folder_mri, "*"))

image, label = save_nii_zoom(input_paths, ref_path)

Pre-affine [[ 9.96478796e-01 -6.34289905e-02 -5.48329167e-02 -1.09191711e+02]
 [ 6.33631870e-02  9.97973025e-01 -2.95466068e-03 -1.11349678e+02]
 [ 5.49098887e-02 -5.30124642e-04  9.98491228e-01 -1.54765289e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Transforming spacing of (256, 196, 256) from [1.         0.99998683 1.        ] to [1. 1. 1.]
New shape: (256, 196, 256)
Post-affine [[ 9.96478796e-01 -6.34298250e-02 -5.48329055e-02 -1.09191711e+02]
 [ 6.33631945e-02  9.97986197e-01 -2.95465998e-03 -1.11349678e+02]
 [ 5.49098961e-02 -5.30131627e-04  9.98491168e-01 -1.54765289e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Done
Pre-affine [[ 9.99060214e-01 -2.24244129e-02  3.71105000e-02 -1.32854523e+02]
 [ 1.80671029e-02  9.93274927e-01  1.13925003e-01 -7.95169830e+01]
 [-3.93996127e-02 -1.13141738e-01  9.92796004e-01 -1.07591080e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Transforming spacing of (256, 196